In [1]:
!pip install tensorflowjs #model 파일 변환을 위한 tenserflowjs 설치

     |████████████████████████████████| 77 kB 2.7 MB/s 


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf 

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import layers # 딥러닝 모델을 위한 라이브러리들

from sklearn.preprocessing import MultiLabelBinarizer # label 인코딩을 위한 라이브러리

from google.colab import files  #csv 파일 colab 상으로 업로드를 위함

In [3]:
seed = 10
np.random.seed(seed)

csvFile = files.upload() #데이터 파일 업로드


Saving squrthi_dataset.csv to squrthi_dataset.csv


In [4]:
import io
data = pd.read_csv(io.BytesIO(csvFile['squrthi_dataset.csv']))
data.describe()

,0_position_x,0_position_y,0_score,1_position_x,1_position_y,1_score,2_position_x,2_position_y,2_score,3_position_x,3_position_y,3_score,4_position_x,4_position_y,4_score,5_position_x,5_position_y,5_score,6_position_x,6_position_y,6_score,7_position_x,7_position_y,7_score,8_position_x,8_position_y,8_score,9_position_x,9_position_y,9_score,10_position_x,10_position_y,10_score,11_position_x,11_position_y,11_score,12_position_x,12_position_y,12_score,13_position_x,13_position_y,13_score,14_position_x,14_position_y,14_score,15_position_x,15_position_y,15_score,16_position_x,16_position_y,16_score,img_height,img_width,pose_score
count,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000,3894.000000
mean,115.204254,51.621229,0.652903,117.001972,48.895282,0.611023,114.548556,49.347920,0.296132,122.261485,46.163150,0.664544,119.456924,46.360062,0.175357,128.380199,63.923630,0.807540,125.543501,62.898597,0.733626,123.751195,93.708477,0.685304,118.494486,92.131807,0.549936,111.990199,105.871009,0.688419,106.848965,103.274628,0.577679,141.938868,117.465016,0.790910,138.214819,117.280877,0.749284,122.004921,141.592550,0.789146,119.817769,141.353684,0.681438,134.877946,186.371150,0.678101,131.798512,185.224557,0.666111,239.365948,236.742938,0.635144
std,57.654731,43.243893,0.374440,59.154220,43.818096,0.395434,58.778138,44.292140,0.268888,56.464872,44.787454,0.380662,57.041492,43.621270,0.211597,55.665469,44.929993,0.309141,54.191146,44.232718,0.294443,58.151730,46.985006,0.317006,54.486493,46.249752,0.284166,59.143838,47.209599,0.316046,56.813188,46.707020,0.317600,55.748364,51.304896,0.302918,54.405760,51.349842,0.287365,56.880864,58.448384,0.292986,55.590716,58.042812,0.275895,56.323233,71.782313,0.282182,54.752752,70.999551,0.274702,70.344511,61.606850,0.238221
min,-10.266961,-13.311756,0.000528,-7.899345,-10.793723,0.000737,-8.486717,-10.557241,0.000509,-7.927922,-15.310076,0.000386,-7.803750,-17.679083,0.000610,-7.514142,-15.425010,0.000674,-11.738182,-14.129022,0.000594,-8.927567,-6.215543,0.000537,-10.103379,-8.580183,0.000576,-9.849440,-5.287575,0.000613,-5.326314,-5.101714,0.000805,-6.494373,-6.097191,0.000802,-6.254012,-7.448987,0.000718,-10.041183,-6.990811,0.001133,-7.021356,-5.702689,0.000901,-6.753867,-0.903168,0.000942,-7.085725,-0.151686,0.000773,224.000000,75.000000,0.001182
25%,82.174680,17.811400,0.287682,84.024055,15.341071,0.144778,79.082299,17.965777,0.080674,92.025356,15.454824,0.270349,84.199487,15.273727,0.048392,96.757452,32.320653,0.813452,96.244143,31.379778,0.655803,81.459693,60.644567,0.458440,80.396671,58.618166,0.337980,63.016596,79.352513,0.484566,55.509663,77.401621,0.285070,121.844717,82.470825,0.847211,115.384364,82.033345,0.756932,83.134951,119.261960,0.790964,85.215053,116.978282,0.561463,106.756041,159.899434,0.667727,108.949807,159.408623,0.647086,224.000000,224.000000,0.582301
50%,99.934886,45.475647,0.871224,102.203510,41.498385,0.858528,98.753556,42.734409,0.231390,110.772324,37.970587,0.907613,105.495848,39.122087,0.085254,117.467633,57.399602,0.963919,113.541477,56.770523,0.857576,124.450696,87.090472,0.829724,107.955270,85.646493,0.569079,109.539688,91.010098,0.828936,102.803144,89.846612,0.683737,138.475872,112.326237,0.921407,131.387271,110.367597,0.854336,100.895249,128.609167,0.932293,100.199696,128.124141,0.758034,123.852179,174.587816,0.791983,122.214633,173.858505,0.784821,224.000000,224.000000,0.721157
75%,143.736

## 데이터 셋 설명
자체 수집한 스쿼트 데이터를 PoseNet 모델을 통해 <br>
0.코   
1.왼쪽 눈  
2.오른쪽 눈  
3.왼쪽 귀  
4.오른쪽 귀  
5.왼쪽 어깨  
6.오른쪽 어깨  
7.왼쪽 팔꿈치  
8.오른쪽 팔꿈치  
9.왼쪽 손목  
10.오른쪽 손목  
11.왼쪽 골반 부위  
12.오른쪽 골반 부위  
13.왼쪽 무릎  
14.오른쪽 무릎  
15.왼쪽 발목  
16.오른쪽 발목  

각각의 x,y 좌표와 신뢰도점수, 이미지 크기와 ,전체 신뢰도 점수를 구해 데이터 셋으로 만들었습니다.  

[출처) 포즈예측 | Tensorflow Lite](https://www.tensorflow.org/lite/examples/pose_estimation/overview?hl=ko)




In [5]:
for i in range(17):
  str_i = str(i)
  data[str_i+'_position_x'] = data[str_i+'_position_x'] / data['img_width']
  data[str_i+'_position_y'] = data[str_i+'_position_y'] / data['img_height']

# 이미지 크기로 각 x,y 좌표를 나누어 이미지 크기에 구애 받지 않는 데이터셋으로 변환
label = data['label']
features = data.drop(['label'],axis=1).drop(['img_width'],axis=1).drop(['img_height'],axis=1) # label과 feature 분리
features

,0_position_x,0_position_y,0_score,1_position_x,1_position_y,1_score,2_position_x,2_position_y,2_score,3_position_x,3_position_y,3_score,4_position_x,4_position_y,4_score,5_position_x,5_position_y,5_score,6_position_x,6_position_y,6_score,7_position_x,7_position_y,7_score,8_position_x,8_position_y,8_score,9_position_x,9_position_y,9_score,10_position_x,10_position_y,10_score,11_position_x,11_position_y,11_score,12_position_x,12_position_y,12_score,13_position_x,13_position_y,13_score,14_position_x,14_position_y,14_score,15_position_x,15_position_y,15_score,16_position_x,16_position_y,16_score,pose_score
0,0.484019,0.217185,0.999630,0.509988,0.189067,0.999089,0.457195,0.193550,0.995371,0.589390,0.172364,0.994726,0.446178,0.187281,0.165161,0.686377,0.288158,0.982096,0.438376,0.308389,0.992269,0.647712,0.373408,0.975397,0.323656,0.381621,0.957905,0.477061,0.413979,0.939079,0.233191,0.438614,0.937968,0.732559,0.545577,0.992528,0.566215,0.542443,0.845789,0.727955,0.652699,0.676873,0.391073,0.617740,0.911579,0.792642,0.903796,0.661586,0.524984,0.860047,0.899776,0.878048
1,0.410678,0.177078,0.943056,0.431595,0.156383,0.939027,0.412172,0.156436,0.195429,0.479352,0.176484,0.976651,0.477914,0.176047,0.083235,0.488943,0.277629,0.978470,0.527140,0.264160,0.768275,0.314167,0.383167,0.975430,0.341586,0.386526,0.416535,0.395467,0.261692,0.663997,0.405780,0.261818,0.347468,0.681547,0.537234,0.906657,0.692983,0.534838,0.826725,0.406507,0.554733,0.954348,0.427595,0.552097,0.587574,0.550363,0.790711,0.777261,0.537665,0.788734,0.856859,0.717470
2,0.420460,0.165984,0.980809,0.442014,0.140702,0.973469,0.421644,0.149711,0.322021,0.489133,0.155788,0.977872,0.485783,0.157976,0.077083,0.501685,0.262109,0.979947,0.532092,0.236187,0.851095,0.324228,0.379334,0.947029,0.337074,0.379113,0.419343,0.375945,0.253868,0.535626,0.396971,0.248658,0.529243,0.660913,0.527092,0.899191,0.695714,0.520667,0.818391,0.409824,0.548447,0.954457,0.422214,0.547548,0.664743,0.540764,0.788204,0.751326,0.538524,0.782021,0.802781,0.734378
3,0.428937,0.165301,0.976938,0.448833,0.141543,0.959500,0.425436,0.148013,0.284211,0.489002,0.161593,0.977440,0.467549,0.156966,0.071706,0.513511,0.255830,0.979377,0.521578,0.238022,0.849731,0.322415,0.370970,0.931279,0.337299,0.362507,0.480947,0.412581,0.252921,0.556498,0.403143,0.237392,0.522214,0.676569,0.531098,0.918189,0.693284,0.521638,0.845587,0.413453,0.544372,0.949601,0.420375,0.542948,0.641696,0.550355,0.786331,0.754817,0.539082,0.784566,0.794513,0.734956
4,0.419368,0.165659,0.984347,0.442869,0.142001,0.975753,0.418391,0.147569,0.325113,0.486615,0.163561,0.975058,0.458658,0.157422,0.071524,0.507858,0.256786,0.983051,0.524339,0.234484,0.886211,0.328343,0.370989,0.957979,0.344026,0.361481,0.400825,0.394296,0.233503,0.551038,0.408835,0.226278,0.488715,0.666091,0.530294,0.904141,0.693725,0.525209,0.842895,0.408898,0.546222,0.967087,0.422957,0.545095,0.537590,0.547445,0.787241,0.737658,0.535459,0.786799,0.814423,0.729612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3889,0.621501,0.115808,0.733553,0.636424,0.054846,0.675106,0.599747,0.030815,0.544589,0.698180,0.033714,0.932795,0.581319,0.054835,0.483606,0.850748,0.199271,0.960986,0.532861,0.247365,0.925901,0.974609,0.386624,0.982274,0.539304,0.461437,0.906746,0.892855,0.513654,0.971579,0.569763,0.513730,0.903934,0.832446,0.615877,0.555415,0.654494,0.587986,0.824549,0.952209,0.608894,0.947548,0.604995,0.682971,0.876941,0.851346,0.978752,0.798295,0.866745,1.010010,0.534789,0.797565
3890,0.601754,0.107321,0.961449,0.640900,0.058446,0.941402,0.587843,0.068592,0.818200,0.711170,0.029253,0.963391,0.554408,0.068348,0.575816,0.847239,0.193752,0.973375,0.530196,0.250539,0.962772,1.001840,0.379647,0.955439,0.542092,0.454441,0.636702,0.929903,0.511515,0.977414,0.570940,0.505936,0.827549,0.267122,0.444593,0.647308,0.646011,0.577582,0.85465

In [6]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(features, label, test_size=0.3)

In [7]:
def build_model():
  model = keras.Sequential([
                            keras.Input(shape = [len(train_x.keys())]),
                            layers.Dense(64, activation='relu'),
                            layers.Dense(64, activation='relu'),
                            layers.Dense(64, activation='relu'),
                            layers.Dense(3, activation = 'softmax')
  ])

  model.compile(
      loss='categorical_crossentropy',
      optimizer = 'adam',
      metrics=['acc'])
  return model

In [8]:
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                3392      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 3)                 195       
                                                                 
Total params: 11,907
Trainable params: 11,907
Non-trainable params: 0
_________________________________________________________________


In [16]:
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(label.str.split(","))# Label one-hot encoding
mlb.classes_ 

array(['nothing', 'squrthi', 'standingside'], dtype=object)

In [17]:
train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size=0.3)

In [ ]:
from sklearn.utils import validation
model.fit(train_x,train_y, epochs = 30)

Epoch 1/30
86/86 [==============================] - 0s 4ms/step - loss: 0.0384 - acc: 0.9875
Epoch 2/30
86/86 [==============================] - 0s 4ms/step - loss: 0.0127 - acc: 0.9963
Epoch 3/30
86/86 [==============================] - 0s 4ms/step - loss: 0.0112 - acc: 0.9960
Epoch 4/30
86/86 [==============================] - 0s 4ms/step - loss: 0.0093 - acc: 0.9960
Epoch 5/30
86/86 [==============================] - 0s 5ms/step - loss: 0.0164 - acc: 0.9945
Epoch 6/30
86/86 [==============================] - 0s 5ms/step - loss: 0.0096 - acc: 0.9956
Epoch 7/30
86/86 [==============================] - 0s 3ms/step - loss: 0.0095 - acc: 0.9971
Epoch 8/30
86/86 [==============================] - 0s 4ms/step - loss: 0.0121 - acc: 0.9956
Epoch 9/30
86/86 [==============================] - 0s 4ms/step - loss: 0.0066 - acc: 0.9978
Epoch 10/30
86/86 [==============================] - 0s 4ms/step - loss: 0.0091 - acc: 0.9960
Epoch 11/30
86/86 [==============================] - 0s 4ms/step - lo

In [ ]:
model.evaluate(test_x, test_y) #테스트 데이터로 검증
mlb.classes_ # 출력 형태

In [ ]:
# 테스트를 위한 코드
# csvTestFile =  files.upload() 
# test = pd.read_csv(io.BytesIO(csvTestFile['csv_downloadfile (9).csv']))
# test
# proba = model.predict(test)
# idx = np.argmax(proba)
# proba

Save-Model


In [ ]:
import tensorflowjs as tfjs #저장된 모델을 불러오기 위함
tfjs.converters.save_keras_model(model,artifacts_dir="./sqrt_model.json")


In [ ]:
model.save('squart_model.h5')